In [11]:
import os
import pandas as pd
from time import time, sleep
from datetime import datetime

import pyautogui

from PIL import Image, ImageOps

import cv2 as cv
import numpy as np

import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'C:\Users\war23\AppData\Local\Tesseract-OCR\tesseract.exe'

# Notes
- underperforms on light backgrounds
- frequently only recognizes 2 of the 3 numbers
- tesseract [single digits, psm, resizing, limited dictionary](https://stackoverflow.com/questions/63572276/pytesseract-not-seeing-some-single-digit-numbers-in-table)

### Known Issues
- recognizing the "teammate dead" skull icon (bottom left) instead of the kill counter

In [2]:
data_dir = 'media/icons/'
out_dir = 'media/test_record_live_stream/'
current_dir = os.getcwd()

light_needle_img = f'light_kills_counter_skull_icon.jpg'  
light_needle_img_2 = f'light_kills_counter_skull_icon_2.jpg' 
dark_needle_img = f'dark_kills_counter_skull_icon.jpg'
dark_needle_img_2 = f'dark_kills_counter_skull_icon_2.jpg'  

armor_plates_icon = 'armor_plates_icon.png' 
armor_satchel_icon = 'armor_satchel_icon.png'
armor_satchel_icon2 = 'armor_satchel_icon2.png'

teams_remaining_icon = 'teams_remaining_icon.png'
players_remaining_icon = 'players_remaining_icon.png'

n_loops = 100
n_top_right_recognized = 0
n_bottom_left_recognized = 0
n_dark_recognized = 0
n_light_recognized = 0
sum_thresh = 0

n_digit_lookup_errors = 0
float_division_by_zero_errors = 0
n_no_digits_recognized_errors = 0

n_bad_pulls = 0

temp_top_right_numbers_stash = []
temp_bottom_left_numbers_stash = []

In [3]:
def capture_screenshot():
    # capture screenshot & resize to 720p
    screenshot = pyautogui.screenshot() 
    screenshot = screenshot.resize((1280, 720))
    
    grayscale = ImageOps.grayscale(screenshot)
    
    # translate to opencv
    screenshot = cv.cvtColor(np.array(grayscale), cv.COLOR_RGB2BGR)
    
    return screenshot


def pull_numbers(image, psm=6):    
    
    image = Image.fromarray(image)
    
    custom_config = f'--psm {psm}  -c tessedit_char_whitelist="012345 6789"'
    # extract string of characters from image 
    image_text = tess.image_to_string(image.resize((int(image.size[0]*1.5), int(image.size[1]*1.5)), Image.ANTIALIAS), config=custom_config)

    # cut all non-digits/spaces, then split into list of ints
    numbers = [int(n) for n in ''.join([c for c in image_text if c in ' 0123456789']).split()]
    
    return numbers


def record_numbers(numbers, file_path):
    existing_df = pd.read_csv(file_path)

    temp_df = pd.DataFrame(numbers)
    temp_df.columns = existing_df.columns
    
    new_df = pd.concat([existing_df, temp_df], axis=0)
    new_df.to_csv(file_path, index=False)
    
    return new_df

In [4]:
import os

#### RESET IMAGE RECORDS
for f in os.listdir(f'{out_dir}bottom_left_numbers'):
    if '.jpg' in f:
        os.remove(f'{out_dir}bottom_left_numbers/{f}')
pd.read_csv(f'{out_dir}bottom_left_numbers/sample_records.csv').head(0).to_csv(f'{out_dir}bottom_left_numbers/sample_records.csv', index=False)

for f in os.listdir(f'{out_dir}top_right_numbers'):
    if '.jpg' in f:
        os.remove(f'{out_dir}top_right_numbers/{f}')
pd.read_csv(f'{out_dir}top_right_numbers/sample_records.csv').head(0).to_csv(f'{out_dir}top_right_numbers/sample_records.csv', index=False)

In [5]:
sleep(10)
# save start time
now = time()

In [6]:
for _ in range(n_loops):
    loop_time = time()
    
    if _ < 10:
        loop = f'loop_00{_}'
    elif 10 <= _ < 100:
        loop = f'loop_0{_}'
    else:
        loop = f'loop_{_}'

    # capture screenshot, resize to 720p & process
    screenshot = capture_screenshot()
    
    # TOP RIGHT NUMBERS -- loop through different needles (try different backgrounds) 
    for img in [dark_needle_img, dark_needle_img_2]:  # light_needle_img, light_needle_img_2,  # (never got any correct)
        
        # focus right side of the picture
        top_right_numbers_screenshot = Image.fromarray(screenshot.copy()).crop((640, 0, 1280, 720))
        top_right_numbers_screenshot = np.array(top_right_numbers_screenshot)
        
        needle_img = cv.imread(f'{data_dir}{img}', cv.IMREAD_UNCHANGED)
        result = cv.matchTemplate(top_right_numbers_screenshot, needle_img, cv.TM_CCOEFF_NORMED)

        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        threshold = 0.82
        # do we have a satasfactory match?
        if max_val >= threshold:
#             print(f'found needle | {str(max_val)[:5]} > {threshold} | {img}')
            n_top_right_recognized += 1
#             sum_thresh += max_val
            print(f'\nloop: {_}')
            
            needle_w = needle_img.shape[1]
            needle_h = needle_img.shape[0]

            # tag top left corner, add width & height to find bottom right corner
            top_left = max_loc  # want rectangle
            
            # black out kill skull icon
            bottom_right = (top_left[0] + (needle_w) - 5, top_left[1] + needle_h)
            cv.rectangle(top_right_numbers_screenshot, top_left, bottom_right, 
                         color=(0, 0, 0), thickness=-1)
            
            # correct bottom right and (TESTING) expand left to just grab all linearly alligned numbers at once
            bottom_right = (top_left[0] + (needle_w * 2), top_left[1] + needle_h)
            top_left = tuple([top_left[0]-125, top_left[1]])  
            
            top_right_numbers_screenshot_2 = top_right_numbers_screenshot[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
            # make sure we still have a screenshot
            if top_right_numbers_screenshot_2.size != 0:
                top_right_numbers_screenshot = top_right_numbers_screenshot_2
            
                # black out squads and players remaining icons
                for img in [teams_remaining_icon, players_remaining_icon]:  # light_needle_img, light_needle_img_2,  # (never got any correct)
                    needle_img_2 = cv.imread(f'{data_dir}{img}', cv.IMREAD_GRAYSCALE)
                    needle_img_2 = cv.cvtColor(needle_img_2, cv.COLOR_RGB2BGR)
                    result_2 = cv.matchTemplate(top_right_numbers_screenshot, needle_img_2, cv.TM_CCOEFF_NORMED)

                    min_val_2, max_val_2, min_loc_2, max_loc_2 = cv.minMaxLoc(result_2)

                    threshold = 0.82
                    # do we have a satasfactory match?
                    if max_val_2 >= threshold:

                        needle_2_w = needle_img_2.shape[1]
                        needle_2_h = needle_img_2.shape[0]

                        # tag top left corner, add width & height to find bottom right corner
                        top_left_2 = max_loc_2  # want rectangle

                        # black out squads and players remaining icons
                        bottom_right_2 = (top_left_2[0] + (needle_2_w) - 5, top_left_2[1] + needle_2_h)
                        cv.rectangle(top_right_numbers_screenshot, top_left_2, bottom_right_2, 
                                     color=(0, 0, 0), thickness=-1)

            out = f'{out_dir}top_right_numbers/{loop}.jpg'
                
            
            # convert opencv back to PIL
            i = Image.fromarray(top_right_numbers_screenshot)
            i.save(out)
            
            # extract numbers (text)
            numbers = pull_numbers(top_right_numbers_screenshot, psm=7)
            
            # make sure we have 3 values
            if len(numbers) < 3:
                for _ in range(3 - len(numbers)):
                    numbers.append(None)
                    
            # add reference file and timestamp to numbers
            numbers.append(out)
            numbers.append(str(datetime.now()))
                    
            temp_top_right_numbers_stash.append(numbers) 

    if (n_loops % 10 == 0) and (len(temp_top_right_numbers_stash) != 0):
        record_numbers(temp_top_right_numbers_stash, f'{out_dir}top_right_numbers/sample_records.csv')
        temp_top_right_numbers_stash = []
                
                
    # BOTTOM LEFT NUMBERS -- loop through different needles (try different backgrounds) 
    for img in [armor_plates_icon, armor_satchel_icon, armor_satchel_icon2]:  # light_needle_img, light_needle_img_2,  # (never got any correct)
        bottom_left_numbers_screenshot = screenshot.copy()
        needle_img = cv.imread(f'{data_dir}{img}')
        result = cv.matchTemplate(bottom_left_numbers_screenshot, needle_img, cv.TM_CCOEFF_NORMED)

        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        threshold = 0.72
        # do we have a satasfactory match?
        if max_val >= threshold:
            n_bottom_left_recognized += 1
            print(f'\n-----\nloop: {_} | {img}')

            needle_w = needle_img.shape[1]
            needle_h = needle_img.shape[0]

            # tag top left corner, add width & height to find bottom right corner
            top_left = max_loc  # want rectangle
            
            # set bottom right and (TESTING) expand left to just grab all linearly alligned numbers at once
            bottom_right = (top_left[0] + (needle_w * 2), top_left[1] + needle_h + 15)
            top_left = tuple([top_left[0]-200, top_left[1]-150])  
            
            bottom_left_numbers_screenshot = bottom_left_numbers_screenshot[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
            
            out =  f'{out_dir}bottom_left_numbers/{loop}.jpg'
                
            # convert opencv back to PIL & save image
            if bottom_left_numbers_screenshot.size != 0:
                temp = []
                
                text = tess.image_to_string(bottom_left_numbers_screenshot)

                temp.append(text)
                
                i = Image.fromarray(bottom_left_numbers_screenshot)
                i.save(out)
                temp.append(out)
                
                temp.append(str(datetime.now()))

                temp_bottom_left_numbers_stash.append(temp) 
            else:
                print(f'ValueError: tile cannot extend outside image\ntop_left = {top_left}\nbottom_right = {bottom_right}\n')
                
    if (n_loops % 10 == 0) and (len(temp_bottom_left_numbers_stash) != 0):
        record_numbers(temp_bottom_left_numbers_stash, f'{out_dir}bottom_left_numbers/sample_records.csv')
        temp_bottom_left_numbers_stash = []
            
    sleep(1)


# print(f'\nDone.\nRecognized: {n_recognized}/{n_loops}\n# Dark recognized: {n_dark_recognized}\n# Light recognized: {n_light_recognized}\nAvg thresh: {sum_thresh / n_recognized}\ndigit_lookup_errors: {n_digit_lookup_errors}')


-----
loop: 2 | armor_satchel_icon2.png

-----
loop: 3 | armor_satchel_icon2.png

loop: 4

-----
loop: 2 | armor_satchel_icon2.png

-----
loop: 5 | armor_satchel_icon2.png

-----
loop: 6 | armor_satchel_icon2.png
ValueError: tile cannot extend outside image
top_left = (20, -136)
bottom_right = (280, 69)


-----
loop: 7 | armor_satchel_icon2.png

-----
loop: 8 | armor_satchel_icon2.png

-----
loop: 9 | armor_satchel_icon2.png

-----
loop: 10 | armor_satchel_icon2.png

loop: 11

-----
loop: 12 | armor_satchel_icon2.png

-----
loop: 14 | armor_satchel_icon2.png

-----
loop: 19 | armor_satchel_icon2.png

loop: 20

-----
loop: 2 | armor_satchel_icon2.png

-----
loop: 21 | armor_satchel_icon2.png
ValueError: tile cannot extend outside image
top_left = (1032, -140)
bottom_right = (1292, 65)


-----
loop: 22 | armor_satchel_icon2.png

-----
loop: 23 | armor_satchel_icon2.png

-----
loop: 24 | armor_satchel_icon2.png
ValueError: tile cannot extend outside image
top_left = (751, -71)
bottom_rig

In [7]:
then = time()
print(f'runtime: {then - now} seconds')

runtime: 226.05324482917786 seconds


## Trying different pytesseract configs on kills only

In [12]:
# import os
# from PIL import Image
# import pytesseract as tess
# tess.pytesseract.tesseract_cmd = r'C:\Users\war23\AppData\Local\Tesseract-OCR\tesseract.exe'

# custom_config = f'--psm 6 -c tessedit_char_whitelist="012345 6789"'
# d = 'media/test_record_live_stream/top_right_numbers/'
# count = 0
# possible = 0

# for pic in os.listdir(d):
#     if '.jpg' in pic:
#         possible += 1
#         k = Image.open(f'{d}{pic}').crop((139,0,167,28))
#         t = tess.image_to_string(k.resize((int(k.size[0]*1.5), int(k.size[1]*1.3)), Image.ANTIALIAS), config=custom_config)
#         if len(t) > 1:
#             print(f'{pic}: {t}')
#             count += 1
#             print()

# print(f'\ncount: {count}')
# print(f'possible: {possible}')

### Testing different resizing on full 

In [10]:
# ii = Image.open(pic)
# ii

In [ ]:
# custom_config = f'--psm 6 -c tessedit_char_whitelist="012345 6789"'
# for x in [0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3]:
#     image_text = tess.image_to_string(ii.resize((int(ii.size[0]*x), int(ii.size[1]*x)), Image.ANTIALIAS), config=custom_config)
#     print(f'x = {x}\nimage_text = {image_text}\n')

### Testing resizing on kills only

In [ ]:
# kills_only = ii.crop((140,0,167,28))
# kills_only

In [ ]:
# custom_config = f'--psm 10 -c tessedit_char_whitelist="012345 6789"'
# image_text = tess.image_to_string(kills_only.resize((100, 100), Image.ANTIALIAS), config=custom_config)
# print(image_text)